In [1]:
%matplotlib widget

In [2]:
import time 
import serial 
from matplotlib import pyplot as plt
import flammkuchen as fl
import numpy as np
from IPython.display import clear_output
from pathlib import Path
from scipy.signal import medfilt,savgol_filter
from scipy import interpolate
from scipy.stats import zscore

In [3]:
def fill_nan(A):
    '''
    interpolate to fill nan values
    '''
    inds = np.arange(A.shape[0])
    good = np.where(np.isfinite(A))
    f = interpolate.interp1d(inds[good], A[good],bounds_error=False)
    B = np.where(np.isfinite(A),A,f(inds))
    return B

In [4]:
ser = serial.Serial('COM7', 9600,timeout=.1) 
ser.close()
ser.open()

In [5]:
# save_path = Path(r"C:\Users\Asus\Desktop\calibrations_valence\new_setup_configuration\pH4.5")
cal_num = 0

In [9]:
tot_recording_s = 90
sleep_int = .05

In [12]:
time_vect = []
values = []
buffer = []
start_exp = time.time()
ongoing = True
while ongoing:
    t = time.time() - start_exp
    
    mystring = ser.readline().decode().rstrip()
    try:
        value = float(mystring)
    except:
        value = np.nan
    time_vect.append(t)
    values.append(value)
    clear_output(wait=True)
    time.sleep(sleep_int)
    print("Sensor Value is", value, "Time", t)
    ser.flush() 
    ser.flushInput() 
    ser.flushOutput()
    if t > tot_recording_s:
        ongoing = False

Sensor Value is nan Time 90.012455701828


In [13]:
plt.figure()
plt.plot(time_vect,fill_nan(np.array(values)))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
plt.figure()
plt.plot(time_vect,fill_nan(np.array(values)))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
times_ps = [0,40,40,80,80,160,160,220,220,260,260,270,270,310,310,420]
pin_status = [0,0,1,1,0,0,0,0,1,1,0,0,1,1,0,0]

In [22]:
all_time = times_ps.copy()
all_pin_status = pin_status.copy()
for _ in range(4):
    all_time.extend(list(np.array(times_ps)+all_time[-1]))
    all_pin_status.extend(pin_status)

In [23]:
sp = np.mean(np.diff(time_vect))

In [43]:
pos_curve  = []
neg_curve = []
pos_ = True
acc = 0
plt.figure()
plt.plot(np.arange(len(values))*sp, zscore(fill_nan(medfilt(values,17))), "k",lw=2)
plt.plot(all_time, all_pin_status)
plt.ylabel("Voltage(V)")
plt.xlabel("Time Points")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
fl.save(save_path/("conductivity_calibration_ph4.5_iti_40secpulse"+str(cal_num)), {"t":time_vect,
                                                             "seonsor_value":values})

In [18]:
ser.close()

In [15]:
80*1.2

96.0

In [16]:
96/50

1.92